### Code reference: 
### [1] Bertagnolli, N (2020). Build a BERT Sci-kit Learn Transformer. 
### http://www.nbertagnolli.com/jekyll/update/2020/03/19/build-a-sklearn-bert-transformer.html

In [ ]:
!pip install transformers

In [7]:
import pandas as pd
import numpy as np
import pickle
import time
import torch
import outlier_detection
from sklearn.ensemble import IsolationForest
from transformers import RobertaTokenizer, RobertaModel
from sklearn.pipeline import Pipeline

## Get cleaned emails without duplicates

In [8]:
with open('no_duplicates_0_95.pkl', 'rb') as f:
    emails = pickle.load(f)

## Pipeline: RobBERT + Isolation Forest

In [5]:
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")
model_RobBERT = RobertaModel.from_pretrained("pdelobelle/robbert-v2-dutch-base")

In [8]:
start_time = time.time()
bert_transformer = outlier_detection.BertTransformer(tokenizer, model_RobBERT)
classifier = IsolationForest(random_state=0, contamination=0.03, n_jobs=1)
model = Pipeline(
    [
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
    ]
)
y_pred = model.fit_predict(emails["only_body"])
print("--- %s seconds ---" % (time.time() - start_time))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


--- 3969.620465517044 seconds ---


In [9]:
emails["pred"] = y_pred
normal = emails.loc[emails['pred'] == 1]
not_outlier_ids = normal.id.tolist()

## Save IDs without outliers

In [102]:
with open('list_ids_for_annotation.pkl', 'wb') as f:
    pickle.dump(not_outlier_ids, f)